In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  7 07:33:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime → "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
path = '/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/layers/seq2seq-chatbot/'
import os
os.chdir(path)
!pwd

/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/layers/seq2seq-chatbot


In [5]:
! pip install -r requirements.txt

     |████████████████████████████████| 368kB 2.8MB/s 
     |████████████████████████████████| 3.3MB 8.7MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [0]:
import tensorflow as tf
import tensorlayer as tl
import numpy as np
from tensorlayer.cost import cross_entropy_seq, cross_entropy_seq_with_mask
from tqdm import tqdm
from sklearn.utils import shuffle
# from data.twitter import data
from tensorlayer.models.seq2seq import Seq2seq
from tensorlayer.models.seq2seq_with_attention import Seq2seqLuongAttention
import os

In [0]:
EN_WHITELIST = '0123456789abcdefghijklmnopqrstuvwxyz ' # space is included in whitelist
EN_BLACKLIST = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\''

In [0]:
limit = {
        'maxq' : 25,
        'minq' : 2,
        'maxa' : 25,
        'mina' : 2
        }

UNK = 'unk'
VOCAB_SIZE = 8000

In [0]:
import random
import nltk
import itertools
from collections import defaultdict
import numpy as np
import pickle

In [10]:
! pwd

/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/layers/seq2seq-chatbot


In [0]:
def get_id2line():
    lines=open('data/movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
    id2line = {}
    for line in lines:
        _line = line.split(' +++$+++ ')
        if len(_line) == 5:
            id2line[_line[0]] = _line[4]
    return id2line

def get_conversations():
    conv_lines = open('data/movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')
    convs = [ ]
    for line in conv_lines[:-1]:
        _line = line.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
        convs.append(_line.split(','))
    return convs

In [0]:
def extract_conversations(convs,id2line,path='data'):
    idx = 0
    for conv in convs:
        f_conv = open(path + str(idx)+'.txt', 'w')
        for line_id in conv:
            f_conv.write(id2line[line_id])
            f_conv.write('\n')
        f_conv.close()
        idx += 1

In [0]:
def gather_dataset(convs, id2line):
    questions = []; answers = []

    for conv in convs:
        if len(conv) %2 != 0:
            conv = conv[:-1]
        for i in range(len(conv)):
            if i%2 == 0:
                questions.append(id2line[conv[i]])
            else:
                answers.append(id2line[conv[i]])

    return questions, answers

In [0]:
def prepare_seq2seq_files(questions, answers, path='data/',TESTSET_SIZE = 30000):

    # open files
    train_enc = open(path + 'train.enc','w')
    train_dec = open(path + 'train.dec','w')
    test_enc  = open(path + 'test.enc', 'w')
    test_dec  = open(path + 'test.dec', 'w')

    # choose 30,000 (TESTSET_SIZE) items to put into testset
    test_ids = random.sample([i for i in range(len(questions))],TESTSET_SIZE)

    for i in range(len(questions)):
        if i in test_ids:
            test_enc.write(questions[i]+'\n')
            test_dec.write(answers[i]+ '\n' )
        else:
            train_enc.write(questions[i]+'\n')
            train_dec.write(answers[i]+ '\n' )
        if i%10000 == 0:
            print('\n>> written {} lines'.format(i))

    # close files
    train_enc.close()
    train_dec.close()
    test_enc.close()
    test_dec.close()

In [0]:
def filter_line(line, whitelist):
    return ''.join([ ch for ch in line if ch in whitelist ])

In [0]:
def filter_data(qseq, aseq):
    filtered_q, filtered_a = [], []
    raw_data_len = len(qseq)

    assert len(qseq) == len(aseq)

    for i in range(raw_data_len):
        qlen, alen = len(qseq[i].split(' ')), len(aseq[i].split(' '))
        if qlen >= limit['minq'] and qlen <= limit['maxq']:
            if alen >= limit['mina'] and alen <= limit['maxa']:
                filtered_q.append(qseq[i])
                filtered_a.append(aseq[i])

    # print the fraction of the original data, filtered
    filt_data_len = len(filtered_q)
    filtered = int((raw_data_len - filt_data_len)*100/raw_data_len)
    print(str(filtered) + '% filtered from original data')

    return filtered_q, filtered_a

In [0]:
def index_(tokenized_sentences, vocab_size):
    # get frequency distribution
    freq_dist = nltk.FreqDist(itertools.chain(*tokenized_sentences))
    # get vocabulary of 'vocab_size' most used words
    vocab = freq_dist.most_common(vocab_size)
    # index2word
    index2word = ['_'] + [UNK] + [ x[0] for x in vocab ]
    # word2index
    word2index = dict([(w,i) for i,w in enumerate(index2word)] )
    return index2word, word2index, freq_dist

In [0]:
def filter_unk(qtokenized, atokenized, w2idx):
    data_len = len(qtokenized)

    filtered_q, filtered_a = [], []

    for qline, aline in zip(qtokenized, atokenized):
        unk_count_q = len([ w for w in qline if w not in w2idx ])
        unk_count_a = len([ w for w in aline if w not in w2idx ])
        if unk_count_a <= 2:
            if unk_count_q > 0:
                if unk_count_q/len(qline) > 0.2:
                    pass
            filtered_q.append(qline)
            filtered_a.append(aline)

    # print the fraction of the original data, filtered
    filt_data_len = len(filtered_q)
    filtered = int((data_len - filt_data_len)*100/data_len)
    print(str(filtered) + '% filtered from original data')

    return filtered_q, filtered_a

In [0]:
def zero_pad(qtokenized, atokenized, w2idx):
    # num of rows
    data_len = len(qtokenized)

    # numpy arrays to store indices
    idx_q = np.zeros([data_len, limit['maxq']], dtype=np.int32)
    idx_a = np.zeros([data_len, limit['maxa']], dtype=np.int32)

    for i in range(data_len):
        q_indices = pad_seq(qtokenized[i], w2idx, limit['maxq'])
        a_indices = pad_seq(atokenized[i], w2idx, limit['maxa'])

        #print(len(idx_q[i]), len(q_indices))
        #print(len(idx_a[i]), len(a_indices))
        idx_q[i] = np.array(q_indices)
        idx_a[i] = np.array(a_indices)

    return idx_q, idx_a

In [0]:
def pad_seq(seq, lookup, maxlen):
    indices = []
    for word in seq:
        if word in lookup:
            indices.append(lookup[word])
        else:
            indices.append(lookup[UNK])
    return indices + [0]*(maxlen - len(seq))

In [0]:
def process_data():
    id2line = get_id2line()
    print('>> gathered id2line dictionary.\n')
    convs = get_conversations()
    print(convs[121:125])
    print('>> gathered conversations.\n')
    questions, answers = gather_dataset(convs,id2line)

    # change to lower case (just for en)
    questions = [ line.lower() for line in questions ]
    answers = [ line.lower() for line in answers ]

    # filter out unnecessary characters
    print('\n>> Filter lines')
    questions = [ filter_line(line, EN_WHITELIST) for line in questions ]
    answers = [ filter_line(line, EN_WHITELIST) for line in answers ]

    # filter out too long or too short sequences
    print('\n>> 2nd layer of filtering')
    qlines, alines = filter_data(questions, answers)

    for q,a in zip(qlines[141:145], alines[141:145]):
        print('q : [{0}]; a : [{1}]'.format(q,a))

    # convert list of [lines of text] into list of [list of words ]
    print('\n>> Segment lines into words')
    qtokenized = [ [w.strip() for w in wordlist.split(' ') if w] for wordlist in qlines ]
    atokenized = [ [w.strip() for w in wordlist.split(' ') if w] for wordlist in alines ]
    print('\n:: Sample from segmented list of words')

    for q,a in zip(qtokenized[141:145], atokenized[141:145]):
        print('q : [{0}]; a : [{1}]'.format(q,a))

    # indexing -> idx2w, w2idx
    print('\n >> Index words')
    idx2w, w2idx, freq_dist = index_( qtokenized + atokenized, vocab_size=VOCAB_SIZE)

    # filter out sentences with too many unknowns
    print('\n >> Filter Unknowns')
    qtokenized, atokenized = filter_unk(qtokenized, atokenized, w2idx)
    print('\n Final dataset len : ' + str(len(qtokenized)))


    print('\n >> Zero Padding')
    idx_q, idx_a = zero_pad(qtokenized, atokenized, w2idx)

    print('\n >> Save numpy arrays to disk')
    # save them
    np.save('data/idx_q.npy', idx_q)
    np.save('data/idx_a.npy', idx_a)

    # let us now save the necessary dictionaries
    metadata = {
            'w2idx' : w2idx,
            'idx2w' : idx2w,
            'limit' : limit,
            'freq_dist' : freq_dist}

    # write to disk : data control dictionaries
    with open('data/metadata.pkl', 'wb') as f:
        pickle.dump(metadata, f)

    # count of unknowns
    unk_count = (idx_q == 1).sum() + (idx_a == 1).sum()
    # count of words
    word_count = (idx_q > 1).sum() + (idx_a > 1).sum()

    print('% unknown : {0}'.format(100 * (unk_count/word_count)))
    print('Dataset count : ' + str(idx_q.shape[0]))


    #print '>> gathered questions and answers.\n'
    #prepare_seq2seq_files(questions,answers)

In [0]:
import numpy as np
from random import sample
def split_dataset(x, y, ratio = [0.9, 0.05, 0.05] ):
    # number of examples
    data_len = len(x)
    lens = [ int(data_len*item) for item in ratio ]

    trainX, trainY = x[:lens[0]], y[:lens[0]]
    testX, testY = x[lens[0]:lens[0]+lens[1]], y[lens[0]:lens[0]+lens[1]]
    validX, validY = x[-lens[-1]:], y[-lens[-1]:]

    return (trainX,trainY), (testX,testY), (validX,validY)

In [0]:
def batch_gen(x, y, batch_size):
    # infinite while
    while True:
        for i in range(0, len(x), batch_size):
            if (i+1)*batch_size < len(x):
                yield x[i : (i+1)*batch_size ].T, y[i : (i+1)*batch_size ].T

In [0]:
def rand_batch_gen(x, y, batch_size):
    while True:
        sample_idx = sample(list(np.arange(len(x))), batch_size)
        yield x[sample_idx].T, y[sample_idx].T

In [0]:
def decode(sequence, lookup, separator=''): # 0 used for padding, is ignored
    return separator.join([ lookup[element] for element in sequence if element ])

In [26]:
! pwd

/content/gdrive/My Drive/Colab Notebooks/cmpe258/final-project/Version-v4/layers/seq2seq-chatbot


In [27]:
process_data()

>> gathered id2line dictionary.

[['L447', 'L448'], ['L490', 'L491'], ['L716', 'L717', 'L718', 'L719', 'L720', 'L721'], ['L750', 'L751', 'L752', 'L753', 'L754', 'L755']]
>> gathered conversations.


>> Filter lines

>> 2nd layer of filtering
28% filtered from original data
q : [you hate me dont you]; a : [i dont really think you warrant that strong an emotion]
q : [then say youll spend dollar night at the track with me]; a : [and why would i do that]
q : [come on  the ponies the flat beer you with money in your eyes me with my hand on your ass]; a : [you  covered in my vomit]
q : [are you following me]; a : [i was in the laundromat i saw your car thought id say hi]

>> Segment lines into words

:: Sample from segmented list of words
q : [['you', 'hate', 'me', 'dont', 'you']]; a : [['i', 'dont', 'really', 'think', 'you', 'warrant', 'that', 'strong', 'an', 'emotion']]
q : [['then', 'say', 'youll', 'spend', 'dollar', 'night', 'at', 'the', 'track', 'with', 'me']]; a : [['and', 'why', 'woul

In [0]:
def load_data(PATH='data/cornell_corpus'):
    # read data control dictionaries
    with open(PATH + 'metadata.pkl', 'rb') as f:
        metadata = pickle.load(f)
    # read numpy arrays
    idx_q = np.load(PATH + 'idx_q.npy')
    idx_a = np.load(PATH + 'idx_a.npy')
    return metadata, idx_q, idx_a

In [0]:
data_corpus = "cornell_corpus"
def initial_setup(data_corpus):
    metadata, idx_q, idx_a = load_data(PATH='data/{}/'.format(data_corpus))
    (trainX, trainY), (testX, testY), (validX, validY) = split_dataset(idx_q, idx_a)
    trainX = tl.prepro.remove_pad_sequences(trainX.tolist())
    trainY = tl.prepro.remove_pad_sequences(trainY.tolist())
    testX = tl.prepro.remove_pad_sequences(testX.tolist())
    testY = tl.prepro.remove_pad_sequences(testY.tolist())
    validX = tl.prepro.remove_pad_sequences(validX.tolist())
    validY = tl.prepro.remove_pad_sequences(validY.tolist())
    return metadata, trainX, trainY, testX, testY, validX, validY

In [0]:
metadata, trainX, trainY, testX, testY, validX, validY = initial_setup(data_corpus)

In [31]:
src_len = len(trainX)
print(src_len)
tgt_len = len(trainY)
print(tgt_len)
assert src_len == tgt_len

86825
86825


In [0]:
batch_size = 32
n_step = src_len // batch_size
src_vocab_size = len(metadata['idx2w']) 
emb_dim = 1024

word2idx = metadata['w2idx'] 
idx2word = metadata['idx2w']

unk_id = word2idx['unk']   
pad_id = word2idx['_']

start_id = src_vocab_size  
end_id = src_vocab_size + 1

word2idx.update({'start_id': start_id})
word2idx.update({'end_id': end_id})
idx2word = idx2word + ['start_id', 'end_id']

src_vocab_size = tgt_vocab_size = src_vocab_size + 2

num_epochs = 50
vocabulary_size = src_vocab_size

In [0]:
def inference(seed, top_n):
  model_.eval()
  seed_id = [word2idx.get(w, unk_id) for w in seed.split(" ")]
  sentence_id = model_(inputs=[[seed_id]], seq_length=20, start_token=start_id, top_n = top_n)
  sentence = []
  for w_id in sentence_id[0]:
      w = idx2word[w_id]
      if w == 'end_id':
          break
      sentence = sentence + [w]
  return sentence

In [34]:
decoder_seq_length = 20
model_ = Seq2seq(
    decoder_seq_length = decoder_seq_length,
    cell_enc=tf.keras.layers.GRUCell,
    cell_dec=tf.keras.layers.GRUCell,
    n_layer=3,
    n_units=256,
    embedding_layer=tl.layers.Embedding(vocabulary_size=vocabulary_size, embedding_size=emb_dim))

[TL] Embedding embedding_1: (8004, 1024)
[TL] RNN rnn_1: cell: GRUCell, n_units: 256
[TL] RNN rnn_2: cell: GRUCell, n_units: 256
[TL] RNN rnn_3: cell: GRUCell, n_units: 256
[TL] RNN rnn_4: cell: GRUCell, n_units: 256
[TL] RNN rnn_5: cell: GRUCell, n_units: 256
[TL] RNN rnn_6: cell: GRUCell, n_units: 256
[TL] Reshape reshape_1
[TL] Dense  dense_1: 8004 No Activation
[TL] Reshape reshape_2
[TL] Reshape reshape_3


In [0]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)
model_.train()

In [0]:
# Uncomment below statements if you have already saved the model
# load_weights = tl.files.load_npz(name='model.npz')
# tl.files.assign_weights(load_weights, model_)

In [0]:
seeds = ["happy birthday have a nice day", 
         "how are you", 
         "I want to have some coffee", 
         "how old are you", 
         "what do you think about coffee",
         "do you like this school",
         "do you want to go shopping"]
for epoch in range(num_epochs):
    model_.train()
    trainX, trainY = shuffle(trainX, trainY, random_state=0)
    total_loss, n_iter = 0, 0
    for X, Y in tqdm(tl.iterate.minibatches(inputs=trainX, targets=trainY, batch_size=batch_size, shuffle=False), 
                    total=n_step, desc='Epoch[{}/{}]'.format(epoch + 1, num_epochs), leave=False):

        X = tl.prepro.pad_sequences(X)
        _target_seqs = tl.prepro.sequences_add_end_id(Y, end_id=end_id)
        _target_seqs = tl.prepro.pad_sequences(_target_seqs, maxlen=decoder_seq_length)
        _decode_seqs = tl.prepro.sequences_add_start_id(Y, start_id=start_id, remove_last=False)
        _decode_seqs = tl.prepro.pad_sequences(_decode_seqs, maxlen=decoder_seq_length)
        _target_mask = tl.prepro.sequences_get_mask(_target_seqs)

        with tf.GradientTape() as tape:
            ## compute outputs
            output = model_(inputs = [X, _decode_seqs])
            
            output = tf.reshape(output, [-1, vocabulary_size])
            ## compute loss and update model
            loss = cross_entropy_seq_with_mask(logits=output, target_seqs=_target_seqs, input_mask=_target_mask)

            grad = tape.gradient(loss, model_.all_weights)
            optimizer.apply_gradients(zip(grad, model_.all_weights))
        
        total_loss += loss
        n_iter += 1

    # printing average loss after every epoch
    print('Epoch [{}/{}]: loss {:.4f}'.format(epoch + 1, num_epochs, total_loss / n_iter))

    for seed in seeds:
        print("Query >", seed)
        top_n = 3
        for i in range(top_n):
            sentence = inference(seed, top_n)
            print(" >", ' '.join(sentence))
    model_path = 'model-cornell-{}.npz'.format(epoch)
    tl.files.save_npz(model_.all_weights, name=model_path)

Epoch [1/50]: loss 5.2160
Query > happy birthday have a nice day
 > i dont want you to see me
 > i know you were in the same
 > i know you were not unk and unk unk
Query > how are you
 > i just dont want you
 > i just dont know what i am was going
 > i know i know what to do i know what to tell you
Query > I want to have some coffee
 > i dont know what i am
 > i dont think i was in the unk
 > i know what i mean is so much i dont want you to do
Query > how old are you
 > i know i dont know
 > i just want to be in the world in a world
 > i dont want you know it is i just want to do
Query > what do you think about coffee
 > i didnt say i know i dont want a favor
 > i know you dont want to get a unk in my unk unk unk and
 > i didnt want to see
Query > do you like this school
 > i dont know i know
 > i didnt
 > i dont want it
Query > do you want to go shopping
 > i know you were unk
 > i dont want you in the car
 > i know i dont want you
[TL] [*] Saving TL weights into model-cornell-0.npz
[

Epoch [2/50]: loss 4.6162
Query > happy birthday have a nice day
 > i think so
 > i think i was unk in a unk unk
 > i think so i didnt know
Query > how are you
 > i think i was a very great man in the unk
 > i think so i dont want you back here for me for
 > i didnt do it
Query > I want to have some coffee
 > i dont know
 > i know what you want to see me in your mouth
 > i dont want to know that i want you too much
Query > how old are you
 > i think so
 > i didnt do anything i want
 > i think so its the unk
Query > what do you think about coffee
 > i didnt do anything to me
 > i think so
 > i didnt know i was
Query > do you like this school
 > i think i had to talk
 > i didnt know
 > i didnt know
Query > do you want to go shopping
 > i didnt want a unk
 > i didnt want you
 > i dont want to talk about this stuff
[TL] [*] Saving TL weights into model-cornell-1.npz
[TL] [*] Saved


Epoch [3/50]: loss 4.3906
Query > happy birthday have a nice day
 > i cant believe you i dont know how to feel
 > i dont know what i did to
 > i cant
Query > how are you
 > i dont think so i dont think i could get it
 > i dont have a lot of money and
 > i just thought you were a little unk and a little bit
Query > I want to have some coffee
 > i think you can do that for you to get a gun on the street unk
 > i dont know i dont think i had a unk unk unk but i was just very grateful
 > i cant
Query > how old are you
 > i am
 > i am not unk
 > i was a little unk
Query > what do you think about coffee
 > i am
 > i dont think i had it in the middle of my house i was just a little
 > i dont know what you want
Query > do you like this school
 > no no
 > no i think
 > no im sorry
Query > do you want to go shopping
 > no i just want a drink with a unk i cant
 > no i didnt have to
 > no i didnt mean to be
[TL] [*] Saving TL weights into model-cornell-2.npz
[TL] [*] Saved


Epoch [4/50]: loss 4.2101
Query > happy birthday have a nice day
 > i dont want to see it was it i need to go to chicago unk and unk
 > i like it
 > i dont want to see you i know i mean it
Query > how are you
 > i think you know i got to go home and get me a drink and get it back
 > i dont think im a good one
 > i am not sure im sure i can go
Query > I want to have some coffee
 > i think i could
 > i cant i cant see
 > i think you should be a unk
Query > how old are you
 > i didnt know you had a little idea
 > i dont know
 > i am
Query > what do you think about coffee
 > i dont know what you want
 > i dont know
 > i dont know i just dont know what you want to say i am unk for unk
Query > do you like this school
 > i think i could have been in the room and
 > i like it im just going to get it up to me i got it
 > i think we should be unk
Query > do you want to go shopping
 > i want a unk unk to the bathroom i dont know
 > i think im gonna have to
 > i dont know what youre talking to me


Epoch [5/50]: loss 4.0470
Query > happy birthday have a nice day
 > i dont think i should be here
 > i think you should be unk
 > i dont know
Query > how are you
 > i dont know i was thinking
 > i was here i didnt
 > i was just doing my unk
Query > I want to have some coffee
 > i cant take the unk i want a beer to unk
 > i dont think so
 > i dont think so much
Query > how old are you
 > i dont know what you mean i am not talking about
 > i dont know what i think i am im just doing that for
 > i am not unk
Query > what do you think about coffee
 > i cant
 > i was unk the guy i dont want
 > i dont know i dont know
Query > do you like this school
 > yeah i think i can do that
 > yeah i think i could have to get it off to her unk
 > yeah i guess
Query > do you want to go shopping
 > i cant take you to my unk i dont know
 > i dont think i should go home and i dont
 > i want a drink
[TL] [*] Saving TL weights into model-cornell-4.npz
[TL] [*] Saved


...

Epoch [24/50]: loss 2.6599
Query > happy birthday have a nice day
 > hey thank you
 > hey nice
 > hey man thanks nice and slow
Query > how are you
 > not good but its true
 > not really i dont care about unk and i dont know what
 > not yet i think i have something right on the phone
Query > I want to have some coffee
 > no problem
 > no i wont
 > no problem i dont get it out of here
Query > how old are you
 > to charge and murder i used here
 > to have the energy waits girl to do something for me
 > to do i am here and 1
Query > what do you think about coffee
 > i dont know what i think
 > i thought you might paste it into something
 > i thought it would be nice
Query > do you like this school
 > yes sir
 > yes i think it would have unk us to unk me
 > yes i have no idea how i feel about that unk and very unk for now
Query > do you want to go shopping
 > i dont think so
 > i have to
 > i think we need some help
[TL] [*] Saving TL weights into model-cornell-23.npz
[TL] [*] Saved


Epoch [25/50]: loss 2.6477
Query > happy birthday have a nice day
 > i have a look to you and you cant
 > i got a call with my mom
 > i got that unk problem
Query > how are you
 > i told ya i brought you back and see the address and everything
 > i must be leaving sir
 > i must have told them how do you recognize your voice
Query > I want to have some coffee
 > no thanks i didnt do anything
 > no way
 > no thanks i dont know
Query > how old are you
 > to beat him i am here
 > to beat him once theyre engaged
 > to breathe ma de unk
Query > what do you think about coffee
 > i dont know i think im an unk surgeon i really am
 > i thought you were unk a few minor unk
 > i dont know i dont think im a little unk strange unk
Query > do you like this school
 > yes but yes i am very much
 > yes i can do this for me myself but i always unk my ass
 > yes but im very proud
Query > do you want to go shopping
 > no im fine
 > no im fine
 > no i dont
[TL] [*] Saving TL weights into model-cornell-24.np

Epoch [26/50]: loss 2.6385
Query > happy birthday have a nice day
 > yeah really
 > yeah i guess you should come with me like the unk daughter
 > yeah i guess
Query > how are you
 > based upon the carpet
 > based right on my mind about this i been excited you will
 > based upon building upon the heart on
Query > I want to have some coffee
 > thanks to the bed tonight tonight
 > thanks to you buddy i dont know what i mean
 > thanks for the ride
Query > how old are you
 > i came over here for a year and
 > i am the borg i guess
 > i came to the unk and they were dark for us
Query > what do you think about coffee
 > i thought you were a fucking brother
 > i was just a group in a hospital here
 > i was unk
Query > do you like this school
 > yes i know im a little boy i know you know
 > yes yes
 > yes but yes
Query > do you want to go shopping
 > no sir
 > no i cant do it later i dont care what im going
 > no sir
[TL] [*] Saving TL weights into model-cornell-25.npz
[TL] [*] Saved


Epoch[27/50]:  57%|█████▋    | 1550/2713 [06:26<04:54,  3.95it/s]

Buffered data was truncated after reaching the output size limit.